# Single Phase Flow Examples

<a id='table'></a>

## Examples

- [Example 1. Boundary conditions](#ex1)
<img src=".\images\ex1.gif">

- [Example 2. Three injectors and one producer](#ex2)
<img src=".\images\ex2.gif">

In [39]:
from modules.simulator.simulator import *
from modules.simulator.two_phase_flow import *
from modules.simulator.simplots import *
from modules.simulator.units import UnitRegistry
import numpy as np

# Set image folder path
imgpath = ".\\images"
# Units
u = UnitRegistry()

<a id='ex1'></a>

In [40]:
# Same fluid for all the examples
# FLUID
cf = 5E-5 / u.psi
miu = lambda p:  1.0 * np.exp(5E-5 * (p/u.psi -2800)) * u.centi * u.poise
rho = lambda p: 800 * np.exp(cf * (p - 15 * u.psi) )  * u.kilogram/ u.meter**3
fvf = lambda p: 1.0 * np.exp(- cf * (p - 15 * u.psi) )   # adimensional
fluid = singleFluid(miu, rho, fvf, cf)

## [Example 1. Boundary conditions](#table)

In this example there is a single rock layer. The layer is 5 cells by 30 cells. It has a constant pressure in the West face of 3000 psi and a pressure of 2900 in the East face. The initial pressure is 3000 psi for all the cells. The simulation is run for 20 timesteps of 0.01 days with the Lagging Coefficient Solver.

In [41]:
# GRID
Nx, Ny, Nz = np.array([5, 30, 1])
Sx, Sy, Sz = np.array([1, 1, 1]) * 0.1 * u.meter
depth = np.hstack((Sz * 1 * np.ones([Nx * Ny, ])))
grid = uniformCartesianGrid(Nx, Ny, Nz, Sx, Sy, Sz, depth)

# ROCK
poro = np.random.random_sample(grid.cellnumber)
perm = 5 * np.random.random_sample(grid.cellnumber) * u.milli * u.darcy
cr = 3E-6 / u.psi
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)

# SCHEDULE
timesteps = 0.01 * np.ones(20) *u.day
sch = Schedule(timesteps)

# SOURCE TERM
source_empty = np.zeros([grid.cellnumber, 1])

# BOUNDARY
boundary = Boundary()
# North side of reservoir has a constant  pressure of 2300 psi
boundary.set_boundary_condition('W', 'constant-pressure', 3000 * u.psi)
boundary.set_boundary_condition('E', 'constant-pressure', 2900 * u.psi)

# Set initial conditions
p_init = 3000 * u.psi * np.ones([grid.cellnumber, ])

# WELLS
# Empty wells term
wells_empty = Wells(grid, rock, fluid)

# Initilize solver
LCM = LaggingCoefficients(
    grid, rock, fluid, wells_empty, source_empty, p_init, boundary, gravity=True)

# Run simulation
r, well_solution, sch = LCM.solve(sch, max_inner_iter = 2, tol = 1E-6, ATS = False)

# Transform units to psi
p = r/u.psi

# We will  save a plot for each timestep and then use 
# ffmpeg to create a gif
acum_sum = np.cumsum(sch.timesteps) / u.day
# Add "0" for the initial conditions
acum_sum = np.hstack((0, acum_sum))

for k in np.arange(0,acum_sum.size, 1): 
    plotCellValues2D(grid, p[:, k], 'inferno', np.min(p), np.max(p),
                     title='Pressure (psi). {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\ex1_{}'.format(imgpath, k))

Solving timestep 1 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 2 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 3 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 4 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 5 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 6 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 7 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 8 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 9 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 10 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 11 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 12 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 13 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 14 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 15 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 16 / 20
	 Inner loop 1 / 2
	 Inner loop 2 / 2
S

<a id='ex2'></a>

## [Example 2. Three injectors and one producer.](#table)

In this example there are three layers of rock, each one with measures of 10 by 10 cells. Each cell is a cube with a side of 1 meter. There are three injectors and one producer well in the bottom layer. The simulation is run for 2 days with the Fully Implicit Method.

In [50]:
# GRID
Nx, Ny, Nz = np.array([10, 10, 3])
Sx, Sy, Sz = np.array([1, 1, 1]) * 1 * u.meter
depth = np.hstack((Sz * 1 * np.ones([Nx * Ny, ]), 
                  Sz * 1 * np.ones([Nx * Ny, ]),
                  Sz * 1 * np.ones([Nx * Ny, ])))

grid = uniformCartesianGrid(Nx, Ny, Nz, Sx, Sy, Sz, depth)

# ROCK
poro = np.random.random_sample(grid.cellnumber)
perm = 10 * np.random.random_sample(grid.cellnumber) * u.milli * u.darcy
cr = 3E-6 / u.psi
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)

#SOURCE
source2 = np.zeros([grid.cellnumber, 1])
source2[25] = -2 * u.barrel/u.hour
source2[50] = -2 * u.barrel/u.hour
source2[9] = -1 * u.barrel/u.hour

# INITIAL CONDITIONS
p_init = 3000 * u.psi * np.ones([grid.cellnumber, ])

# --- WELLS
wells = Wells(grid, rock, fluid)
wells.add_vertical_well(0.35 * u.feet, 99, 2500 * u.psi, 0, 'Producer 1' )

# --- Initilize simulator
# SCHEDULE
timesteps = 0.01 * np.ones(200) *u.day
sch = Schedule(timesteps)

FIM  = ImplicitAnalytic(
    grid, rock, fluid, wells, source2, p_init, empty_boundary, gravity=True)

r, well_solution = FIM.solve(sch)

# Transform units to psi
p = r/u.psi

# We will  save a plot for each timestep and then use 
# ffmpeg to create a gif
acum_sum = np.cumsum(sch.timesteps) / u.day
acum_sum = np.hstack((0, acum_sum))

for k in np.arange(0, acum_sum.size, 1):
    plotCellValues2D(grid, p[:, k], 'inferno', np.min(p), np.max(p),
                     title='Pressure (psi). {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\ex2_{}'.format(imgpath, k))

Solving timestep 1  / 200
 	 Newton-Raphson solver :  1/10. Error: 2.02E-02
 	 Newton-Raphson solver :  2/10. Error: 9.40E-06
 	 Newton-Raphson solver :  3/10. Error: 1.19E-09
 	 Newton-Raphson solver :  4/10. Error: 1.79E-13
Solving timestep 2  / 200
 	 Newton-Raphson solver :  1/10. Error: 1.11E-02
 	 Newton-Raphson solver :  2/10. Error: 2.52E-06
 	 Newton-Raphson solver :  3/10. Error: 1.06E-10
 	 Newton-Raphson solver :  4/10. Error: 9.50E-15
Solving timestep 3  / 200
 	 Newton-Raphson solver :  1/10. Error: 8.76E-03
 	 Newton-Raphson solver :  2/10. Error: 1.05E-06
 	 Newton-Raphson solver :  3/10. Error: 6.45E-11
 	 Newton-Raphson solver :  4/10. Error: 9.67E-15
Solving timestep 4  / 200
 	 Newton-Raphson solver :  1/10. Error: 7.83E-03
 	 Newton-Raphson solver :  2/10. Error: 7.49E-07
 	 Newton-Raphson solver :  3/10. Error: 6.20E-11
 	 Newton-Raphson solver :  4/10. Error: 3.20E-15
Solving timestep 5  / 200
 	 Newton-Raphson solver :  1/10. Error: 7.33E-03
 	 Newton-Raphson so

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


<a id="ex3" ></a>

In [53]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# I am using the style sheet found here  Lorena Barba https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css